# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей.

**Цель:** построить модель для определения стоимости. 

## Подготовка данных

### Импорт библиотек и общая информация

In [1]:
import pandas as pd 
import numpy as np 
import lightgbm as lgb
import warnings
import time 

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from catboost import CatBoostRegressor, Pool

warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("D:/Users/Egor/Desktop/Proga/Jupyter/autos/autos.csv")
data.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
data.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [5]:
data.duplicated().sum()

4

**Вывод:**
* Необходимо привести названия столбцов к *snake case*
* Столбцы 'date_crawled', 'registration_month', 'date_created', 'number_of_pictures', 'postal_code', 'last_seen' неинформативны и их можно удалить
* Необходимо удалить дубликаты
* В столбце *price* прусутству.т значения, равные нулю. Скорее всего, эти объекты придется удалить
* В количественных и категориальных признаках присутвуют большое количетсво пропусков, их нужно будет обработать

### Предобработка данных

In [6]:
data.columns = (data.columns
                .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
                .str.lower())
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   kilometer           354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  repaired            283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

In [7]:
data = data.drop(['date_crawled', 'registration_month', 'date_created', 'number_of_pictures', 'postal_code', 'last_seen'], axis=1)
data.head()

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,fuel_type,brand,repaired
0,480,NaN,1993,manual,0,golf,150000,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no


In [8]:
data = data.drop_duplicates()
data.duplicated().sum()

0

In [9]:
print("Процент автомобилей с нулевой ценой:", "{:.2%}".format(len(data.query('price == 0'))/len(data)))

Процент автомобилей с нулевой ценой: 2.88%


In [10]:
data = data.query('price != 0')
print("Процент автомобилей с нулевой ценой:", "{:.2%}".format(len(data.query('price == 0'))/len(data)))

Процент автомобилей с нулевой ценой: 0.00%


In [11]:
data.value_counts('vehicle_type')

vehicle_type
sedan          76513
small          66008
wagon          55081
bus            25455
convertible    17893
coupe          14234
suv            10596
other           2975
dtype: int64

In [12]:
data.value_counts('model')

model
golf                  22946
other                 22282
3er                   16224
polo                  10153
corsa                  9459
                      ...  
kalina                    7
rangerover                4
serie_3                   3
range_rover_evoque        1
serie_1                   1
Length: 250, dtype: int64

In [13]:
data.value_counts('fuel_type')

fuel_type
petrol      183143
gasoline     83470
lpg           4759
cng            513
hybrid         216
other          166
electric        87
dtype: int64

In [14]:
data.value_counts('gearbox')

gearbox
manual    226914
auto       58090
dtype: int64

In [15]:
data.value_counts('repaired')

repaired
no     208476
yes     31630
dtype: int64

In [16]:
print("Процент пропущенных данных для категориальных (небинарных) признаков:")
print("vehicle_type:", "{:.2%}".format(len(data.loc[data['vehicle_type'].isna()])/len(data)))
print("model:", "{:.2%}".format(len(data.loc[data['model'].isna()])/len(data)))
print("fuel_type:", "{:.2%}".format(len(data.loc[data['fuel_type'].isna()])/len(data)))

Процент пропущенных данных для категориальных (небинарных) признаков:
vehicle_type: 10.54%
model: 5.55%
fuel_type: 9.34%


In [17]:
data['vehicle_type'] = data['vehicle_type'].fillna('unknown')
data['model'] = data['model'].fillna('unknown')
data['fuel_type'] = data['fuel_type'].fillna('unknown')

In [18]:
print("Процент пропущенных данных для категориальных (бинарных) признаков:")
print("gearbox:", "{:.2%}".format(len(data.loc[data['gearbox'].isna()])/len(data)))
print("repaired:", "{:.2%}".format(len(data.loc[data['repaired'].isna()])/len(data)))

Процент пропущенных данных для категориальных (бинарных) признаков:
gearbox: 5.13%
repaired: 20.08%


Замена пропущенных значений в gearbox на самые часто встречающиеся:

In [19]:
data['gearbox'] = data['gearbox'].fillna(data.groupby('model')['gearbox'].transform(lambda x: x.value_counts().idxmax()))

Слишком большой процент пропусков (пятая часть), скорее всего данные не заполнялись, потому что машина не проходила ремонт,
заполняем 'no':

In [20]:
data['repaired'] = data['repaired'].fillna('no')

In [21]:
data.isna().sum()

price                0
vehicle_type         0
registration_year    0
gearbox              0
power                0
model                0
kilometer            0
fuel_type            0
brand                0
repaired             0
dtype: int64

In [22]:
data.describe()

,price,registration_year,power,kilometer
count,300425.000000,300425.000000,300425.000000,300425.000000
mean,4619.921098,2004.244916,111.722247,127307.464425
std,4565.205287,83.004128,199.047743,38220.756977
min,1.000000,1000.000000,0.000000,5000.000000
25%,1200.000000,1999.000000,69.000000,125000.000000
50%,2900.000000,2003.000000,105.000000,150000.000000
75%,6650.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


In [23]:
perc = np.arange(0,1,0.05)

In [24]:
data['registration_year'].describe(percentiles=perc)

count    300425.000000
mean       2004.244916
std          83.004128
min        1000.000000
0%         1000.000000
5%         1992.000000
10%        1995.000000
15%        1997.000000
20%        1998.000000
25%        1999.000000
30%        2000.000000
35%        2001.000000
40%        2001.000000
45%        2002.000000
50%        2003.000000
55%        2004.000000
60%        2005.000000
65%        2006.000000
70%        2007.000000
75%        2008.000000
80%        2009.000000
85%        2010.000000
90%        2012.000000
95%        2016.000000
max        9999.000000
Name: registration_year, dtype: float64

Оставим автомобили с годом выпуска от 1992 до 2022:

In [25]:
data = data.query('registration_year >= 1992 and registration_year < 2022')

In [26]:
data['power'].describe(percentiles = perc)

count    287107.000000
mean        112.773914
std         196.426115
min           0.000000
0%            0.000000
5%            0.000000
10%           0.000000
15%          55.000000
20%          60.000000
25%          72.000000
30%          75.000000
35%          86.000000
40%          92.000000
45%         101.000000
50%         105.000000
55%         114.000000
60%         118.000000
65%         126.000000
70%         139.000000
75%         143.000000
80%         150.000000
85%         170.000000
90%         184.000000
95%         218.000000
max       20000.000000
Name: power, dtype: float64

Средняя лошадиная сила машины составляет 212, можно смело убирать все, что превышают 400, заменим нули на медиану по бренду:

In [27]:
data = data.query('power <= 400')
data['power'] = data['power'].replace(0, np.NaN).fillna(data.groupby('model')['power'].transform('median'))

In [28]:
data.describe()

,price,registration_year,power,kilometer
count,286469.000000,286469.000000,286469.000000,286469.000000
mean,4633.407555,2004.137017,119.226625,127904.083862
std,4557.919187,6.059441,51.431578,37517.355945
min,1.000000,1992.000000,0.000000,5000.000000
25%,1200.000000,2000.000000,80.000000,125000.000000
50%,2950.000000,2004.000000,110.000000,150000.000000
75%,6700.000000,2008.000000,145.000000,150000.000000
max,20000.000000,2019.000000,400.000000,150000.000000


**Вывод:**
- Изменены названия столбцов на *snake_case*
- Были удалены дубликаты и неинформативные столбцы
- Пропущенные данные для категориальных (небинарных) признаков были заполнены значением 'unknown'
- Пропущенные значение в *gearbox* заменены на самые часто встречающиеся
- В столбце *repaired* было принято решение заменить пропущенные на 'no'
- Удалены автомобили с неправдоподобным годом выпуска, осталсиь от 1992 до 2022
- Также удалена неправдоподобная лошадиная сила (больше 400), нули заменены на медиану по бренду

## Обучение моделей

### Подготовка данных

In [29]:
data_ohe = data.copy()
data_train, data_test = train_test_split(data_ohe, test_size=0.4, random_state=12345) 

features_train = data_train.drop(['price'], axis=1)
target_train = data_train['price']

features = data_test.drop('price', axis=1)
target = data_test['price']
features_valid, features_test, target_valid, target_test = train_test_split(features, target, test_size=0.5, random_state=12345)

numeric = ['registration_year', 'power', 'kilometer']
categoric = ['brand', 'model', 'vehicle_type', 'gearbox', 'fuel_type', 'repaired']

print(f'Размерность признаков - {features_train.shape} и целевого признака - {target_train.shape} обучающей выборки')
print(f'Размерность признаков - {features_valid.shape} и целевого признака - {target_valid.shape} валидационной выборки')
print(f'Размерность признаков - {features_test.shape} и целевого признака - {target_test.shape} тестовой выборки')

Размерность признаков - (171881, 9) и целевого признака - (171881,) обучающей выборки
Размерность признаков - (57294, 9) и целевого признака - (57294,) валидационной выборки
Размерность признаков - (57294, 9) и целевого признака - (57294,) тестовой выборки


In [30]:
features_valid_cat, features_test_cat, features_train_cat = features_valid.copy(), features_test.copy(), features_train.copy()
for i in categoric:
    features_valid_cat[i] = features_valid_cat[i].astype('category').cat.codes
    features_test_cat[i] = features_test_cat[i].astype('category').cat.codes
    features_train_cat[i] = features_train_cat[i].astype('category').cat.codes

In [31]:
col_transformer = make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='ignore'),
        categoric
    ),
    (
        StandardScaler(), 
        numeric
    ),
    remainder='passthrough',
    verbose_feature_names_out=False
)

features_train = pd.DataFrame.sparse.from_spmatrix(
    col_transformer.fit_transform(features_train),
    columns=col_transformer.get_feature_names_out()
)


features_test = pd.DataFrame.sparse.from_spmatrix(
    col_transformer.transform(features_test),
    columns=col_transformer.get_feature_names_out()
)

features_valid = pd.DataFrame.sparse.from_spmatrix(
    col_transformer.transform(features_valid),
    columns=col_transformer.get_feature_names_out()
)

### Регрессия LGBM

In [32]:
%%time
lgbmmodel = LGBMRegressor() 
param = [{'num_leaves':[50, 100], 'learning_rate':[.1, .3], 'random_state':[12345]}]
clf = GridSearchCV(lgbmmodel, param, scoring='neg_mean_squared_error')
clf.fit(features_train, target_train)
print(clf.best_params_)

{'learning_rate': 0.3, 'num_leaves': 100, 'random_state': 12345}
Wall time: 1min 45s


In [33]:
%%time
lgbmmodel = LGBMRegressor(learning_rate=.3, num_leaves=100, random_state=12345)
lgbmmodel.fit(features_train, target_train)

Wall time: 1.99 s


LGBMRegressor(learning_rate=0.3, num_leaves=100, random_state=12345)

In [34]:
%%time
lgbm_predict = lgbmmodel.predict(features_valid)

Wall time: 338 ms


In [35]:
mse_lgbmmodel = mean_squared_error(target_valid, lgbm_predict)
print("RMSE для LGBM", mse_lgbmmodel ** 0.5)

RMSE для LGBM 1580.4355183057166


### Регрессия DecisionTree

In [36]:
%%time
trmodel = DecisionTreeRegressor(criterion='mse', max_depth=10,  random_state=12345) 
trmodel.fit(features_train, target_train)

Wall time: 3.39 s


DecisionTreeRegressor(criterion='mse', max_depth=10, random_state=12345)

In [37]:
%%time
trmodel_predict = trmodel.predict(features_valid)

Wall time: 55 ms


In [38]:
mse_trmodel = mean_squared_error(target_valid, trmodel_predict)
print("RMSE для DecisionTree:", (mse_trmodel) ** 0.5)

RMSE для DecisionTree: 1978.8385148434074


### Регрессия CatBoost

In [39]:
%%time
catmodel = CatBoostRegressor(loss_function='RMSE')
parameters_cat = {'depth':[5,10], 'learning_rate':np.arange(0.1,1,0.2)}
catmodel_grid = catmodel.grid_search(parameters_cat,
            Pool(features_train_cat, target_train, cat_features=categoric),
            cv=3, verbose=True, plot=False)

0:	learn: 5967.8824306	test: 5955.7330666	best: 5955.7330666 (0)	total: 262ms	remaining: 4m 22s
1:	learn: 5499.4923536	test: 5490.4316282	best: 5490.4316282 (1)	total: 379ms	remaining: 3m 9s
2:	learn: 5092.6203394	test: 5084.4546675	best: 5084.4546675 (2)	total: 501ms	remaining: 2m 46s
3:	learn: 4716.7965090	test: 4711.0706953	best: 4711.0706953 (3)	total: 617ms	remaining: 2m 33s
4:	learn: 4396.8126027	test: 4391.3722794	best: 4391.3722794 (4)	total: 723ms	remaining: 2m 23s
5:	learn: 4111.5885971	test: 4107.1307873	best: 4107.1307873 (5)	total: 849ms	remaining: 2m 20s
6:	learn: 3851.5372429	test: 3846.8498377	best: 3846.8498377 (6)	total: 948ms	remaining: 2m 14s
7:	learn: 3634.9533706	test: 3630.6114868	best: 3630.6114868 (7)	total: 1.04s	remaining: 2m 9s
8:	learn: 3430.6533968	test: 3426.9079196	best: 3426.9079196 (8)	total: 1.16s	remaining: 2m 7s
9:	learn: 3248.1955192	test: 3244.9546904	best: 3244.9546904 (9)	total: 1.27s	remaining: 2m 5s
10:	learn: 3096.6404929	test: 3091.9807784	b

In [40]:
catmodel_grid['params']

{'depth': 10, 'learning_rate': 0.1}

In [41]:
%%time
final_catmodel = CatBoostRegressor(depth=catmodel_grid['params']['depth'], 
                              learning_rate=catmodel_grid['params']['learning_rate'],
                              loss_function='RMSE',verbose=100)
final_catmodel.fit(Pool(features_train_cat, target_train, cat_features=categoric))

0:	learn: 4229.4541836	total: 228ms	remaining: 3m 47s
100:	learn: 1597.2571781	total: 22.9s	remaining: 3m 24s
200:	learn: 1522.9944979	total: 45.3s	remaining: 3m
300:	learn: 1474.9533824	total: 1m 9s	remaining: 2m 41s
400:	learn: 1441.3687311	total: 1m 33s	remaining: 2m 20s
500:	learn: 1413.9001683	total: 1m 58s	remaining: 1m 57s
600:	learn: 1393.2718332	total: 2m 22s	remaining: 1m 34s
700:	learn: 1372.6102403	total: 2m 46s	remaining: 1m 11s
800:	learn: 1356.3462049	total: 3m 11s	remaining: 47.6s
900:	learn: 1341.2198698	total: 3m 36s	remaining: 23.8s
999:	learn: 1327.6253715	total: 4m 1s	remaining: 0us
Wall time: 4min 4s


In [42]:
%%time
cat_predict = final_catmodel.predict(features_valid_cat)

Wall time: 639 ms


In [43]:
mse_catmodel = mean_squared_error(target_valid, cat_predict)
print("RMSE для CatBoost:", (mse_catmodel) ** 0.5)

RMSE для CatBoost: 1871.8619089015183


## Анализ моделей

In [44]:
df = [["3.39 s", "55 ms", 1978.84],
        ["4min 4s", "639 ms", 1871.86],
        ["1.99 s", "338 ms", 1580.43]]
model = ["DecisionTreeRegressor", "CatBoostRegressor", "LGBMRegressor"]

In [45]:
pd.DataFrame(data=df, index=model, columns=["fit_time", "predict_time", "RMSE"])

,fit_time,predict_time,RMSE
DecisionTreeRegressor,3.39 s,55 ms,1978.84
CatBoostRegressor,4min 4s,639 ms,1871.86
LGBMRegressor,1.99 s,338 ms,1580.43


In [46]:
%%time
lgbm_predict_test = lgbmmodel.predict(features_test)
print("RMSE для LGBM по тестовой выборке", mean_squared_error(target_test, lgbm_predict_test) ** 0.5)

RMSE для LGBM по тестовой выборке 1598.93565815033
Wall time: 355 ms


**Вывод:**

На предобработке данных были:
- Изменены названия столбцов на *snake_case*
- Удалены дубликаты и неинформативные столбцы
- Пропущенные данные для категориальных (небинарных) признаков были заполнены значением 'unknown'
- Пропущенные значение в *gearbox* заменены на самые часто встречающиеся
- В столбце *repaired* было принято решение заменить пропущенные на 'no'
- Удалены автомобили с неправдоподобным годом выпуска, осталсиь от 1992 до 2022
- Также удалена неправдоподобная лошадиная сила (больше 400), нули заменены на медиану по бренду


1. Лучшей моделью стала LGBMRegressor с RMSE равной 1580.43 и параметрами *num_leaves=100* и *learning_rate = 0.3*.
2. CatBoostRegressor с RMSE равной 1871.86 и параметрами *depth = 10* и *learning_rate = 0.1*. Она сильно проигрывает во времени обучения и предсказания другим моделям.
3. DecisionTreeRegressor уступает другим моделям в метрике RMSE = 1978.84, но выигрывает по времени предсказания.

Заказчику важны: качество предсказания, скорость предсказания, время обучения. Исходя из этого можно предложить модель LGBMRegressor. У нее самое выское качаество предсказания и быстрое время обучения и предсказания.